In this homework, we'll deploy the dino or dragon model we trained in the previous homework.

Download the model from here:

https://github.com/SVizor42/ML_Zoomcamp/releases/download/dino-dragon-model/dino_dragon_10_0.899.h5
Question 1

Now convert this model from Keras to TF-Lite format.

In [1]:
# !wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/dino-dragon-model/dino_dragon_10_0.899.h5

In [2]:
import numpy as np
import tensorflow as tf
from io import BytesIO
from urllib import request
from PIL import Image
import requests

2022-11-29 02:03:15.219987: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 02:03:15.316221: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-29 02:03:15.318607: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-29 02:03:15.318619: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [3]:
model_name = "dino_dragon_10_0.899.h5"
tf_model = tf.keras.models.load_model(model_name)

converter = tf.lite.TFLiteConverter.from_keras_model(tf_model)
tflite_model = converter.convert()

tflite_model_name = 'dino_dragon_10_0.899.tflite'
with open(tflite_model_name, 'wb') as f:
    f.write(tflite_model)

2022-11-29 02:03:16.260382: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-29 02:03:16.260402: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-29 02:03:16.260421: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (3511): /proc/driver/nvidia/version does not exist
2022-11-29 02:03:16.260597: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


INFO:tensorflow:Assets written to: /tmp/tmp6ilc6txe/assets


INFO:tensorflow:Assets written to: /tmp/tmp6ilc6txe/assets
2022-11-29 02:03:17.224214: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-11-29 02:03:17.224246: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-11-29 02:03:17.224734: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp6ilc6txe
2022-11-29 02:03:17.225454: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-11-29 02:03:17.225473: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp6ilc6txe
2022-11-29 02:03:17.227681: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2022-11-29 02:03:17.228305: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-11-29 02:03:17.281013: I tensorflow/cc/saved_model/loader.cc:213] Running initializatio

#### Q1. What's the size of the converted model?

In [4]:
!ls -lh | grep '.tflite'

-rw-r--r-- 1 anks anks  43M Nov 29 02:03 dino_dragon_10_0.899.tflite


#### 43 MB

In [5]:
interpreter = tf.lite.Interpreter(tflite_model_name)
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


#### Q2. To be able to use this model, we need to know the index of the input and the index of the output.

What's the output index for this model?

In [6]:
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

print(output_index)

13


#### 13

Preparing the image

You'll need some code for downloading and resizing images. You can use this code:

In [7]:
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.Resampling.NEAREST)  # Image.NEAREST
    return img

Let's download and resize this image:

https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg

Based on the previous homework, what should be the target size for the image?

In [8]:
# !wget https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg

In [9]:
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg"
img = download_image(url)

target_size = (150, 150)
img = prepare_image(img, target_size)

img = np.array(img, dtype=np.float32)
img = np.array([img])
img = img * 1./255

Based on the previous homework, what should be the target size for the image?

#### Q3. Now we need to turn the image into numpy array and pre-process it.

    Tip: Check the previous homework. What was the pre-processing we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

In [10]:
# r, _, _ = img.getpixel((0, 0))
# print(r)

round(img[0][0][0][0], 7)

0.5529412

#### 0.5529412

#### Q4. Now let's apply this model to this image. What's the output of the model?

In [11]:
interpreter.set_tensor(input_index, img)
interpreter.invoke()
output_data = interpreter.get_tensor(output_index)
print(output_data[0][0])

0.82448614


#### 0.82448614

Prepare the lambda code

Now you need to copy all the code into a separate python file. You will need to use this file for the next two questions.

Tip: you can test this file locally with ipython or Jupyter Notebook by importing the file and invoking the function from this file.

Docker

For the next two questions, we'll use a Docker image that we already prepared. This is the Dockerfile that we used for creating the image:

FROM public.ecr.aws/lambda/python:3.9
COPY dino-vs-dragon-v2.tflite .

And pushed it to svizor42/zoomcamp-dino-dragon-lambda:v2.

    Note: The image already contains a model and it's not the same model as the one we used for questions 1-4.

Download the base image svizor42/zoomcamp-dino-dragon-lambda:v2.

#### Q5. So what's the size of this base image?

In [12]:
!docker pull svizor42/zoomcamp-dino-dragon-lambda:v2

v2: Pulling from svizor42/zoomcamp-dino-dragon-lambda
Digest: sha256:46be01f7953cd48ebe526329ddf781d92019b9d44de4ecb67d11f56eaccf79cb
Status: Image is up to date for svizor42/zoomcamp-dino-dragon-lambda:v2
docker.io/svizor42/zoomcamp-dino-dragon-lambda:v2


In [13]:
!docker images svizor42/zoomcamp-dino-dragon-lambda:v2 | awk '{ print $7 }'


639MB


#### 639 MB

Now let's extend this docker image, install all the required libraries and add the code for lambda.

You don't need to include the model in the image. It's already included. The name of the file with the model is dino-vs-dragon-v2.tflite and it's in the current workdir in the image (see the Dockerfile above for the reference).

Now run the container locally.

Score this image: https://upload.wikimedia.org/wikipedia/en/e/e9/GodzillaEncounterModel.jpg

#### Q6. What's the output from the model?

    0.12
    0.32
    0.52
    0.72

In [14]:
url = "http://localhost:5555/2015-03-31/functions/function/invocations"

# data = {
#     'url': 'https://upload.wikimedia.org/wikipedia/en/e/e9/GodzillaEncounterModel.jpg',
#     'model_name': 'dino-vs-dragon-v2.tflite'
# }
data = {'url': 'https://upload.wikimedia.org/wikipedia/en/e/e9/GodzillaEncounterModel.jpg'}

result = requests.post(url, json=data).json()
print(result)

ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

#### 